In [1]:
from pytorch_utils import transform_data4ResNet, transform2tensors, transform4ConvTran
import torch
from models.ConvTran.utils import Setup, Initialization
from models.ConvTran.hyper_parameters import params as transform_params
from models.ConvTran.Models.model import ConvTran
from models.ConvTran.Training import SupervisedTrainer, train_runner


ModuleNotFoundError: No module named 'torch'

In [4]:
# return value has to be False if you didn't install all the nvidia dependecies to use the GPU
torch.cuda.is_available()

False

In [5]:
def instantiate_convTran( n_classes, test_X, test_y, train_X, train_y,device):
    
    # config is a dictionary containing all the default hyper-parameters for ConvTran
    config = {'data_path': 'Dataset/UEA/', 'output_dir': 'Results', 'Norm': False, 'val_ratio': 0.2,
	          'print_interval': 10, 'Net_Type': ['C-T'], 'emb_size': 16, 'dim_ff': 256, 'num_heads': 8,
	          'Fix_pos_encode': 'tAPE', 'Rel_pos_encode': 'eRPE', 'epochs': 100, 'batch_size': 16,
	          'lr': 0.001, 'dropout': 0.01, 'val_interval': 2, 'key_metric': 'accuracy', 'gpu': 0,
	          'console': False, 'seed': None}

    device = Initialization(config,dev=device)
    
    # loading data and initializing the model
    train_loader, test_loader, enc = transform4ConvTran(config, n_classes, test_X, test_y, train_X, train_y)
    model = ConvTran(config, num_classes=config['num_labels']).to(device)
    
    # set optimizer and instantiate train and vali loader
    config['optimizer'] = torch.optim.Adam(model.parameters())
    trainer = SupervisedTrainer(model, train_loader, device, config['loss_module'], config['optimizer'], l2_reg=0,
                                print_interval=config['print_interval'], console=config['console'],
                                print_conf_mat=False)
    val_evaluator = SupervisedTrainer(model, test_loader, device, config['loss_module'],
                                      print_interval=config['print_interval'], console=config['console'],
                                      print_conf_mat=False)
    return config, device, model, trainer, val_evaluator

using a sample dataset used in the sktime documentation

In [6]:
from sktime.datasets import load_basic_motions
from  sktime.datatypes._panel._convert import from_nested_to_3d_numpy

In [7]:
X_train, y_train = load_basic_motions(split="train")
X_test, y_test = load_basic_motions(split="test") 
X_train = from_nested_to_3d_numpy(X_train)
X_test = from_nested_to_3d_numpy(X_test)

In [8]:
print(X_train.shape , X_test.shape, type(X_train), type(X_test),)

(40, 6, 100) (40, 6, 100) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


it is important to use the function from_nested_to_3d_numpy transforming a pandas dataframe into a numpy array ( I didn't test with pandas )

In [9]:
n_classes = X_train.shape[1]
config, device, model, trainer, val_evaluator = instantiate_convTran(n_classes=n_classes, test_X=X_test, test_y=y_test, train_X=X_train, train_y=y_train,device="cpu")

2023-11-28 10:51:08,958 | INFO : Using device: cpu
/home/davide/miniconda3/envs/MTSC_exp_comp/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# this code is currently saving the model into a file, in case you put a non-valid 
# file/ path you're gonna get an error
saving_file = "tmp.pt"
train_runner(config, model, trainer, val_evaluator,saving_file)